# Deep Learning NLP with Pytorch   Final Project 

## Install the correct librairies

Here we install and upgrade all the librairies for this project

In [1]:
!pip install --upgrade plotly pandas-profiling[notebook]
!pip install plotly pandas-profiling[notebook]
#!pip install torch==1.6.0
#!pip install torchtext==0.7.015
#!pip install --upgrade fast_tabnet fastai plotly pandas-profiling[notebook]

     |████████████████████████████████| 13.2MB 8.1MB/s 
     |████████████████████████████████| 245kB 53.4MB/s 
     |████████████████████████████████| 1.1MB 49.2MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 3.2MB 51.7MB/s 
     |████████████████████████████████| 122kB 59.3MB/s 
     |████████████████████████████████| 296kB 52.9MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=7976c89ed29ae958814aa828a97a8bf708f82503ea0aa9a9360ad5bc94d6e323
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256=b71de330cc577144d262c26590f9307bc6df7f7fea2a8be1b5b51a2cfa1416e2
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d856

In [2]:
!pip install nltk

Import the librairies


In [3]:
import os
import re
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import operator
from datetime import date, datetime
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from scipy import stats
from nltk.corpus import stopwords
from collections import Counter
import pathlib
import gc
import string
from pandas_profiling import ProfileReport
import plotly.express as px
pd.options.display.max_columns = 1500
pd.options.plotting.backend = "plotly"

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
#deal with tensors
import torch   
#handling text data
from torchtext import data 

#for attention LSTM
from torch.autograd import Variable

## Import NLTK and  sentiment Analyzer to label the Reditt data 

In [6]:
import nltk as nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download(['words','punkt','vader_lexicon','stopwords','wordnet'])



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk import word_tokenize, corpus
from  nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS

english_words = set(corpus.words.words())
import spacy as sp
nlps = sp.load('en')
import random

In [8]:
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Load the Data from Google Drive 




In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [118]:

path=pathlib.Path("/content/drive/My Drive/reddit_wsb.csv")

data = pd.read_csv(path)

In [119]:
data.head()

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56


## Pre-process DATA

In [120]:
data = data.drop(columns=['id','score','comms_num','url', 'created','timestamp'])
data.head()

,title,body
0,"It's not about the money, it's about sending a...",NaN
1,Math Professor Scott Steiner says the numbers ...,NaN
2,Exit the system,The CEO of NASDAQ pushed to halt trading “to g...
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,NaN
4,"Not to distract from GME, just thought our AMC...",NaN


In [122]:
#data = data['title'].dropna()

In [123]:
data.shape

(42924, 2)

In [124]:
df_copy=data

In [125]:
df_copy.shape

(42924, 2)

In [126]:
ticker_list =["NOK", "GME", "AMC"]

In [127]:
#is_GME = df_copy['title'].str.contains("GME")
#is_AMC =df_copy['title'].str.contains("AMC")
#is_NOK =df_copy['title'].str.contains("NOK")

Create a dataframe that only has NOK, AMC and GME posts

In [128]:

#df_NGA = df_copy[np.logical_or(df_copy['title'].str.contains("GME"),df_copy['title'].str.contains("AMC"))]
#data=df_NGA

In [129]:
#data=df_NGA

In [130]:
#data_other =df_copy

split the data into the title data and the body data




In [131]:
title_data = data[['title']].copy()
body_data = data[['body']].copy()
body_data = body_data.dropna()
title_data = title_data.dropna()

## preprocessing using pytorch

In [132]:
title_data.shape


(42924, 1)

In [133]:
title_data.head()

,title
0,"It's not about the money, it's about sending a..."
1,Math Professor Scott Steiner says the numbers ...
2,Exit the system
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...
4,"Not to distract from GME, just thought our AMC..."


In [134]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [135]:
def clean_text_data(text):
    text = text.lower()

    # Replacing Handlers with Empty String
    text = re.sub('@[^\s]+','',text)

    # Replacing URLs with Empty String
    text = re.sub(r"http\S+", "",text)

    # Remove all the special characters
    text = ' '.join(re.findall(r'\w+', text))

    # Replacing Single Characters with Empty String
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Removing Extra Spaces
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Tokenize into words
    text = word_tokenize(text)

    # Word Lemmatization
    lem = WordNetLemmatizer()
    text = ' '.join([lem.lemmatize(w) for w in text])
    # return text
    tokens = []
    for token in text.split():
      if token not in stop_words:
        tokens.append(stemmer.stem(token))
    return " ".join(tokens)
# Text Preprocessing

title_data.title = title_data.title.apply(lambda x : clean_text_data(x))
body_data.body   = body_data.body.apply(lambda x : clean_text_data(x))

In [136]:
title_data.head()

,title
0,money send messag
1,math professor scott steiner say number spell ...
2,exit system
3,new sec file gme someon le retard pleas interpret
4,distract gme thought amc brother awar


In [137]:
body_data.head()

,body
2,ceo nasdaq push halt trade give investor chanc...
6,hedgefund whale spread disinfo say friday make...
7,life fair mother alway told would complain arb...
10,believ right one rare opportun help good compa...
12,guy champ gme would thought bunch crazi retard...


In [138]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('basic_english')

In [139]:
counter = Counter()
for _, row in title_data.iterrows():
    counter.update(tokenizer(row['title']))

vocab_title = Vocab(counter, min_freq=1)

In [140]:

print(f"Size of TEXT vocabulary: {len(vocab_title)}\n")

print(f"Commonly used words: {vocab_title.freqs.most_common(10)}\n")

Size of TEXT vocabulary: 13778

Commonly used words: [('gme', 8386), ('hold', 5242), ('buy', 4843), ('robinhood', 3212), ('amc', 3195), ('stock', 2570), ('go', 2514), ('sell', 2512), ('fuck', 1985), ('share', 1884)]



In [141]:
counter = Counter()
for _, row in body_data.iterrows():
    counter.update(tokenizer(row['body']))

vocab_body = Vocab(counter, min_freq=1)

In [142]:

print(f"Size of TEXT vocabulary: {len(vocab_body)}\n")

print(f"Commonly used words: {vocab_body.freqs.most_common(10)}\n")

Size of TEXT vocabulary: 34927

Commonly used words: [('stock', 17383), ('share', 16350), ('short', 15449), ('gme', 14335), ('go', 13692), ('buy', 13496), ('like', 12520), ('market', 12246), ('wa', 12109), ('price', 11889)]



### Label the data using NLTK sentiment analyzer

In [143]:
# Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [144]:
# Will generate a score in the format: {'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877} 
def sentiment_analyzer_scores(sentence):
    score = analyzer.polarity_scores(sentence)
    if score['compound'] > 0.05:
        return str('positive')
    elif score['compound'] < -0.05:
        return str('negative')
    else:
        return str('neutral')

In [145]:
# Will generate a score in the format: {'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877} 
def sentiment_analyzer_scores_number(sentence):
    score = analyzer.polarity_scores(sentence)
    if score['compound'] > 0.05:
        return float(1)
    elif score['compound'] < -0.05:
        return float(-1)
    else:
        return float(0)

In [146]:
# Lets create a function to create a list of sentiment values 

def sentiment_append_number(text):
    moods = []
    for mess in text:
        mood = sentiment_analyzer_scores_number(mess)
        moods.append(mood)
    return moods
    

In [147]:
# Add title moods to data frame
title_moods = sentiment_append_number(title_data['title'])
#title_data['title_sentiment'] = title_moods

In [148]:
title_data['title_sentiment'] = title_moods
title_data.rename(columns = {'title_sentiment':'LABEL'}, inplace = True)
title_data.rename(columns = {'title':'TEXT'}, inplace = True)

In [149]:
#title_data = title_data.drop(title_data.columns[2], axis=1, inplace=True)

In [150]:
title_data.head()

,TEXT,LABEL
0,money send messag,0.0
1,math professor scott steiner say number spell ...,1.0
2,exit system,0.0
3,new sec file gme someon le retard pleas interpret,-1.0
4,distract gme thought amc brother awar,-1.0


In [152]:

# Lets find out how many of each sentiment there are in the titles text

title_data['LABEL'].value_counts()

 0.0    20791
 1.0    12408
-1.0     9725
Name: LABEL, dtype: int64

In [153]:
# Add body moods to data frame
body_moods = sentiment_append_number(body_data['body'])
#body_data['title_sentiment'] = body_moods

In [154]:
body_data['body_sentiment'] = body_moods
body_data.rename(columns = {'body_sentiment':'LABEL'}, inplace = True)
body_data.rename(columns = {'body':'TEXT'}, inplace = True)

In [155]:
body_data.head()

,TEXT,LABEL
2,ceo nasdaq push halt trade give investor chanc...,1.0
6,hedgefund whale spread disinfo say friday make...,-1.0
7,life fair mother alway told would complain arb...,-1.0
10,believ right one rare opportun help good compa...,1.0
12,guy champ gme would thought bunch crazi retard...,1.0


In [156]:
body_data['LABEL'].value_counts()

 1.0    10746
-1.0     6247
 0.0     3638
Name: LABEL, dtype: int64

### **PYTORCH** LSTM



In [157]:

#Reproducing same results
SEED = 2315

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True

In [158]:
training_data=body_data

In [159]:
training_data.head()

,TEXT,LABEL
2,ceo nasdaq push halt trade give investor chanc...,1.0
6,hedgefund whale spread disinfo say friday make...,-1.0
7,life fair mother alway told would complain arb...,-1.0
10,believ right one rare opportun help good compa...,1.0
12,guy champ gme would thought bunch crazi retard...,1.0


In [160]:

training_data['length'] = training_data['TEXT'].apply(lambda s: len(s.split()))
training_data.sort_values(by=['length'], ascending=False, inplace=True)


In [161]:
training_data['length'].hist()

In [162]:
training_data = training_data[(training_data['length'] >= 2) & (training_data['length'] <= 700)]



### Split the training data set into train,calidation and test set


In [163]:

from sklearn.model_selection import train_test_split
train, test = train_test_split(training_data, test_size=0.3)

In [164]:
train, valid = train_test_split(train, test_size=0.2)

In [165]:
train.sort_values(by=['length'], ascending=False, inplace=True)
test.sort_values(by=['length'], ascending=False, inplace=True)
valid.sort_values(by=['length'], ascending=False, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [166]:
title_data.rename(columns = {'TEXT':'question_text'}, inplace = True)
title_data.rename(columns = {'LABEL':'target'}, inplace = True)

In [167]:
body_data.rename(columns = {'TEXT':'question_text'}, inplace = True)
body_data.rename(columns = {'LABEL':'target'}, inplace = True)

In [168]:
test.rename(columns = {'TEXT':'question_text'}, inplace = True)
test.rename(columns = {'LABEL':'target'}, inplace = True)
train.rename(columns = {'TEXT':'question_text'}, inplace = True)
train.rename(columns = {'LABEL':'target'}, inplace = True)
valid.rename(columns = {'TEXT':'question_text'}, inplace = True)
valid.rename(columns = {'LABEL':'target'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [169]:
test

,question_text,target,length
32855,edit almost 10 sinc wrote dd steadi 3 4 day ne...,1.0,698
38538,key takeaway short squeez acceler stock price ...,1.0,697
39067,prefac entertain financi advic companion piec ...,1.0,686
5516,hey guy sure go make modblock go long know bea...,1.0,680
23203,felt wa time gather thought shamaran sometim f...,1.0,678
...,...,...,...
7249,fuck hold,-1.0,2
5587,sndl moon,0.0,2
7141,get lower,-1.0,2
28156,obvious retard,-1.0,2


## PYTORCH LSTM

In [182]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch.utils.data import Dataset

class PandasDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        return self.dataframe.iloc[index]

def generate_batch(batch):
    label_list, text_list, text_length_list = [], [], []
    SEQSIZE = len(batch[0]['question_text'].split())
    for row in batch:
        _text = row['question_text']
        _label = row['target']
        label_list.append(label_pipeline(_label))
        processed_text = text_pipeline(_text)[:SEQSIZE]
        text_length_list.append(len(processed_text))
        processed_text += [1]*(SEQSIZE-len(processed_text))
        text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.float32)
    text_list = torch.tensor(text_list, dtype=torch.int64)
    text_length_list = torch.tensor(text_length_list, dtype=torch.int64)
    return label_list.to(device), text_list.to(device), text_length_list


In [183]:
batch_size = 64

dataloader_train = DataLoader(PandasDataset(train), batch_size=batch_size, shuffle=False, collate_fn=generate_batch)

dataloader_test = DataLoader(PandasDataset(test), batch_size=batch_size, shuffle=False, collate_fn=generate_batch)

dataloader_valid = DataLoader(PandasDataset(valid), batch_size=batch_size, shuffle=False, collate_fn=generate_batch)

In [185]:
training_data.head()

,TEXT,LABEL,length
36496,hi guy follow inform uncl bruce latest youtub ...,1.0,698
32470,dear member hello fellow degener gambler may i...,1.0,698
32855,edit almost 10 sinc wrote dd steadi 3 4 day ne...,1.0,698
19138,gon na long stori unfair observ endur 1 corpor...,1.0,698
38538,key takeaway short squeez acceler stock price ...,1.0,697


In [186]:
text_pipeline = lambda x: [vocab_body[token] for token in tokenizer(x)]

label_pipeline = lambda x: int(x)

In [187]:

from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch.utils.data import Dataset

In [188]:
device

device(type='cuda')

In [189]:
len(dataloader_train)

170

In [190]:
len(dataloader_test)

91

In [191]:
len(dataloader_valid)

43

## LSTM Classifier

In [195]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=num_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, 
                                                            text_lengths,
                                                            batch_first=True,
                                                            enforce_sorted=False)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)

        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]

        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

instantiate an LSTM classifier model




In [214]:

#define hyperparameters
size_of_vocab = len(vocab)
embedding_dim = 100
hidden_dim = 16
output_dim = 1
num_layers = 2
bidirection = True
dropout = 0.3

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, hidden_dim, output_dim, num_layers, 
                   bidirectional = True, dropout = dropout)

In [215]:
#architecture
print(model)

#No. of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

# #Initialize the pretrained embedding  ()
# pretrained_embeddings = TEXT.vocab.vectors
# # model.embedding.weight.data.copy_(pretrained_embeddings) # TODO PLEASE USE THIS FOR LSTM

# print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(34927, 100)
  (lstm): LSTM(100, 16, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 3,514,237 trainable parameters


In [216]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [217]:
device

device(type='cuda')

**MODEL TRAIN FUNCTION**

In [218]:
def train(model, dataloader, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for idx, (label, text, text_lengths) in enumerate(dataloader):
        
        #resets the gradients after every batch
        optimizer.zero_grad()   

        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()
    
        #compute the loss
        loss = criterion(predictions, label)
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
        if idx % 1000 == 0:
            print (f"Completed {idx}/{len(dataloader)}")
    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

Model Evaluate function 

In [219]:
def evaluate(model, dataloader, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for _, (label, text, text_lengths) in enumerate(dataloader):
        
            #predict
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

Check  Model Forward Pass

In [220]:
#Check model device type
next(model.parameters()).is_cuda, device

(True, device(type='cuda'))

In [221]:
len(dataloader_train)

170

In [222]:

for idx, (label, text, text_lengths) in enumerate(dataloader_train):
    print ("text.shape: ", text.shape)
    predictions = model(text, text_lengths)
    print ("predictions.shape: ", predictions.shape)
    break

text.shape:  torch.Size([64, 698])
predictions.shape:  torch.Size([64, 1])


In [ ]:

text[0], predictions[0][:10], label[:10]

In [223]:
model_path = 'saved_weights.pt'

N_EPOCHS = 15
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    ts_string = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
    print(f'\n {ts_string} Epoch: {epoch}')
    #train the model
    train_loss, train_acc = train(model, dataloader_train, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, dataloader_valid, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_path)

    print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


 03/26/2021 17:13:47 Epoch: 0
Completed 0/170
	 Train Loss: 0.418 | Train Acc: 35.12%
	 Val. Loss: 0.966 |  Val. Acc: 14.37%

 03/26/2021 17:13:52 Epoch: 1
Completed 0/170
	 Train Loss: 0.257 | Train Acc: 21.77%
	 Val. Loss: 0.623 |  Val. Acc: 16.58%

 03/26/2021 17:13:58 Epoch: 2
Completed 0/170
	 Train Loss: -0.667 | Train Acc: 41.71%
	 Val. Loss: 0.630 |  Val. Acc: 20.62%

 03/26/2021 17:14:04 Epoch: 3
Completed 0/170
	 Train Loss: -1.785 | Train Acc: 47.08%
	 Val. Loss: 0.005 |  Val. Acc: 25.45%

 03/26/2021 17:14:10 Epoch: 4
Completed 0/170
	 Train Loss: -3.238 | Train Acc: 50.61%
	 Val. Loss: -1.077 |  Val. Acc: 29.87%

 03/26/2021 17:14:15 Epoch: 5
Completed 0/170
	 Train Loss: -4.346 | Train Acc: 50.24%
	 Val. Loss: -2.917 |  Val. Acc: 41.06%

 03/26/2021 17:14:21 Epoch: 6
Completed 0/170
	 Train Loss: -6.023 | Train Acc: 53.77%
	 Val. Loss: -3.861 |  Val. Acc: 44.47%

 03/26/2021 17:14:27 Epoch: 7
Completed 0/170
	 Train Loss: -7.024 | Train Acc: 56.41%
	 Val. Loss: -4.346 | 